In [1]:
from transformers import BartTokenizer, TFBartForConditionalGeneration
from transformers.models.bart.modeling_bart import shift_tokens_right
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [2]:
# Set up the T5 tokenizer
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
#model = TFBartForConditionalGeneration.from_pretrained('facebook/bart-base')

In [3]:
df = pd.read_csv('D:/Kuliah/S2/Tesis/Code/Generative-ABSA-main/data/quadruplet/rest15/all_data.csv')
X = df['input'].values.tolist()
y = df['quadruplet_label'].values.tolist()
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
def make_data(x, y, pad_id):
    input_encodings = dict(tokenizer(x, padding=True, truncation=True, return_tensors="pt"))
    target_encodings = dict(tokenizer(y, padding=True, truncation=True, return_tensors="pt"))
    labels = target_encodings['input_ids']
    #decoder_start_token_id=0 biar <s> yang nandain start dari tokenisasi
    decoder_input_ids = shift_tokens_right(labels, pad_id, 0)
    labels = labels.masked_fill_(labels == pad_id, -100)
    encodings = {
        'input_ids': tf.convert_to_tensor(input_encodings['input_ids'].numpy()),
        'attention_mask': tf.convert_to_tensor(input_encodings['attention_mask'].numpy()),
        'decoder_input_ids': tf.convert_to_tensor(decoder_input_ids.numpy()),
        'labels': tf.convert_to_tensor(labels.numpy()),
    }
    return encodings

In [5]:
pad_id = model.config.pad_token_id

NameError: name 'model' is not defined

In [73]:
train_encodings = make_data(x_train, y_train, pad_id)
test_encodings = make_data(x_test, y_test, pad_id)

In [74]:
train_encodings

{'input_ids': <tf.Tensor: shape=(656, 56), dtype=int64, numpy=
 array([[    0,   133,  5765, ...,     1,     1,     1],
        [    0,   133, 28287, ...,     1,     1,     1],
        [    0,   133,  3202, ...,     1,     1,     1],
        ...,
        [    0,   133,   813, ...,     1,     1,     1],
        [    0,  2387, 25537, ...,     1,     1,     1],
        [    0, 24989,  3958, ...,     1,     1,     1]], dtype=int64)>,
 'attention_mask': <tf.Tensor: shape=(656, 56), dtype=int64, numpy=
 array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]], dtype=int64)>,
 'decoder_input_ids': <tf.Tensor: shape=(656, 77), dtype=int64, numpy=
 array([[   0,    0, 1640, ...,    1,    1,    1],
        [   0,    0, 1640, ...,    1,    1,    1],
        [   0,    0, 1640, ...,    1,    1,    1],
        ...,
        [   0,    0, 1640, ...,   

In [75]:
save_path = 'tf-bart-base-quadruplet'
checkpoint_filepath = save_path + "/" + "T5-{epoch:04d}-{val_loss:.4f}.ckpt"
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

callbacks = [model_checkpoint_callback] 
metrics = [tf.keras.metrics.SparseTopKCategoricalAccuracy(name='accuracy') ]

In [76]:
# Define the fine-tuning parameters
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
model.compile(optimizer=optimizer, metrics=metrics)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [83]:
# Fine-tune the model
history = model.fit(train_encodings, epochs=50, batch_size=8, validation_data=test_encodings, callbacks=callbacks)

Epoch 1/50
82/82 [==============================] - ETA: 0s - loss: 0.4945 - accuracy: 0.0309

INFO:tensorflow:Assets written to: tf-bart-base-quadruplet\T5-0001-0.2956.ckpt\assets


INFO:tensorflow:Assets written to: tf-bart-base-quadruplet\T5-0001-0.2956.ckpt\assets


82/82 [==============================] - 68s 836ms/step - loss: 0.4945 - accuracy: 0.0309 - val_loss: 0.2956 - val_accuracy: 0.0491
Epoch 2/50
82/82 [==============================] - ETA: 0s - loss: 0.3639 - accuracy: 0.0361

INFO:tensorflow:Assets written to: tf-bart-base-quadruplet\T5-0002-0.2814.ckpt\assets


...

INFO:tensorflow:Assets written to: tf-bart-base-quadruplet\T5-0004-0.2467.ckpt\assets


82/82 [==============================] - 63s 771ms/step - loss: 0.2147 - accuracy: 0.0319 - val_loss: 0.2467 - val_accuracy: 0.0749
Epoch 5/50
82/82 [==============================] - 14s 170ms/step - loss: 0.1792 - accuracy: 0.0341 - val_loss: 0.2530 - val_accuracy: 0.0724
Epoch 6/50
82/82 [==============================] - 14s 168ms/step - loss: 0.1473 - accuracy: 0.0310 - val_loss: 0.2522 - val_accuracy: 0.0724
Epoch 7/50
82/82 [==============================] - 14s 169ms/step - loss: 0.1225 - accuracy: 0.0325 - val_loss: 0.2493 - val_accuracy: 0.0727
Epoch 8/50
82/82 [==============================] - 14s 170ms/step - loss: 0.1079 - accuracy: 0.0382 - val_loss: 0.2556 - val_accuracy: 0.0492
Epoch 9/50
82/82 [==============================] - 14s 171ms/step - loss: 0.0899 - accuracy: 0.0349 - val_loss: 0.2692 - val_accuracy: 0.0726
Epoch 10/50
82/82 [==============================] - 14s 171ms/step - loss: 0.0744 - accuracy: 0.0342 - val_loss: 0.2640 - val_accuracy: 0.0492
Epoch 11/

INFO:tensorflow:Assets written to: tf-bart-base-quadruplet\T5-0011-0.2421.ckpt\assets


INFO:tensorflow:Assets written to: tf-bart-base-quadruplet\T5-0011-0.2421.ckpt\assets


82/82 [==============================] - 66s 816ms/step - loss: 0.0667 - accuracy: 0.0334 - val_loss: 0.2421 - val_accuracy: 0.0492
Epoch 12/50
82/82 [==============================] - 14s 169ms/step - loss: 0.0651 - accuracy: 0.0359 - val_loss: 0.2682 - val_accuracy: 0.0792
Epoch 13/50
82/82 [==============================] - 14s 167ms/step - loss: 0.0529 - accuracy: 0.0341 - val_loss: 0.2822 - val_accuracy: 0.0513
Epoch 14/50
82/82 [==============================] - 14s 168ms/step - loss: 0.0474 - accuracy: 0.0366 - val_loss: 0.3022 - val_accuracy: 0.0726
Epoch 15/50
82/82 [==============================] - 14s 168ms/step - loss: 0.0496 - accuracy: 0.0333 - val_loss: 0.2973 - val_accuracy: 0.0726
Epoch 16/50
82/82 [==============================] - 14s 167ms/step - loss: 0.0454 - accuracy: 0.0350 - val_loss: 0.2898 - val_accuracy: 0.0725
Epoch 17/50
82/82 [==============================] - 14s 168ms/step - loss: 0.0322 - accuracy: 0.0348 - val_loss: 0.2981 - val_accuracy: 0.0724
Epoc

In [84]:
model.save_pretrained(save_path)

In [78]:
import random

In [92]:
idx = random.randint(0, len(x_test))
input_text =  x_test[idx]
encoded_query = dict(tokenizer(input_text, padding=True, truncation=True, return_tensors="tf"))
input_ids = encoded_query["input_ids"]
attention_mask = encoded_query["attention_mask"]
generated_answer = model.generate(input_ids, attention_mask=attention_mask)
decoded_answer = tokenizer.decode(generated_answer.numpy()[0])

print(x_test[idx])
print(y_test[idx])
print(decoded_answer)

Overall , decent food at a good price , with friendly people 
(food, decent, POS, food prices);(people, friendly, POS, service general)
</s><s>(food, decent, NEU, food prices);(people, friendly, POS</s>
